As the title of this notebook suggests, you may need to display masks over images during competitions or otherwise. Here are some simple functions that can help you do this.

Displaying masks on top of images is not very complicated in itself. However, this can quickly turn into a puzzle for the beginner and waste precious time which takes him away from his main concern.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
import cv2


### **1. Data import**

First of all, for the needs of this notebook we need to load images as well as information about the masks to display. To do this I chose to use the data from the "cloud organization" competition.

As shown in the output of the code above, an image is associated with several RLE codes (in this case one per label). We will have to transform these RLE codes into masks.
Finally, note that an RLE code can be "NaN", ie the class corresponding to the label is not present in the image.

In [ ]:
repBase = '../input/understanding_cloud_organization'
repTrain = '../input/understanding_cloud_organization/train_images'

In [ ]:
df = pd.read_csv(os.path.join(repBase, 'train.csv'))
df['image'] = df['Image_Label'].apply(lambda x: x.split('_')[0])
df['label'] = df['Image_Label'].apply(lambda x: x.split('_')[1])
df = df.drop(['Image_Label'], axis=1)
df.head()

### **2. From RLE to mask**

Still for the needs of the notebook we define below a simple function in order to convert an RLE encoding into a mask.

A mask is made up of 0's and 1's, so it's relatively easy to describe it in a much more compact form than a matrix. This is the goal of RLE encoding (run-length encoding). For more details : https://fr.wikipedia.org/wiki/Run-length_encoding

In [ ]:
def rleToMask(rle: str, shape: tuple  =(1400, 2100)) -> np.ndarray:
    """
    Converting an RLE encoding to a mask

     Parameter
     ----------
     rle   : RLE encoding to convert 
     shape : mask shape

     Return
     ----------
     np.array : mask
    """

    width, height = shape[:2]
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
        current_position += lengths[index]
        
    return mask.reshape(height, width).T

### **3. Mask over image**

We first define some constants, in particular the colors associated with each of the classes as well as the name of the image on which we are going to work.
Following this we load our image then extract the list of RLE codes associated with it.

In [ ]:
colors = [(0,0,255), (255,0,0), (0,255,0), (255,255,0)]
image_name = '7405a00.jpg'

rles = df[df['image']==image_name]['EncodedPixels'].reset_index(drop=True)
image_start = plt.imread(os.path.join(repTrain, image_name))

Here is our original image :

In [ ]:
fig, ax = plt.subplots()
ax.imshow(image_start) 
plt.show()

#### 3.1. Colored mask

For our first example, we are going to display the masks transparently over the image. Each time, we take the image resulting from the previous superposition as support.

In [ ]:
def maskInColor(image : np.ndarray, 
                mask : np.ndarray, 
                color : tuple = (0,0,255), 
                alpha : float=0.2) -> np.ndarray:
    """
    Overlay mask on image

     Parameter
     ----------
     image : image on which we want to overlay the mask 
     mask  : mask to process
     color : color we want to apply on mask
     alpha : opacity coefficient

     Return
     ----------
     np.array : result of layering
    """
    
    image = np.array(image)
    H,W,C = image.shape
    mask    = mask.reshape(H,W,1)
    overlay = image.astype(np.float32)
    overlay =  255-(255-overlay)*(1-mask*alpha*color/255 )
    overlay = np.clip(overlay,0,255)
    overlay = overlay.astype(np.uint8)
    return overlay

Here is the result :

This method is efficient and very visual, it has the advantage of respecting the mask since the reader can distinguish its exact shape. The problem, if any, can appear when two masks overlap. You will notice in this case a mixture of colors which is sometimes difficult to read.

In [ ]:
fig, ax = plt.subplots()

image = np.copy(image_start)         #reset the working image
for k in range(4):                   #We have 4 classes in this dataset
    rle = rles[k]                    #initialize the current RLE code
    if not isinstance(rle, float):   #it's not a 'NaN' RLE
        mask = rleToMask(rles[k])
        image = maskInColor(image, mask, color=colors[k], alpha=0.3)

ax.imshow(image) 
plt.show()

#### 3.2. Bounding box

In our second example we are going to draw a frame (bounding box) around the mask.

In [ ]:
def trace_boundingBox(image : np.ndarray,
                      mask : np.ndarray,
                      color : tuple = (0,0,255),
                      width : int = 10):
    """
    Draw a bounding box on image

     Parameter
     ----------
     image : image on which we want to draw the box 
     mask  : mask to process
     color : color we want to use to draw the box edges
     width : box edges's width

    """
    
    lbl = label(mask)
    props = regionprops(lbl)
    for prop in props:
        coin1 = (prop.bbox[3], prop.bbox[2])
        coin2 = (prop.bbox[1], prop.bbox[0])
        cv2.rectangle(image, coin2, coin1, color, width)

The resulting image is clear. In addition there is no more problem with the superposition of 2 masks. However, unlike the previous example, we no longer distinguish the exact shape of the mask.

In [ ]:
fig, ax = plt.subplots()

image = np.copy(image_start)         #reset the working image
for k in range(4):                   #We have 4 classes in this dataset
    rle = rles[k]                    #initialize the current RLE code
    if not isinstance(rle, float):   #it's not a 'NaN' RLE
        mask = rleToMask(rles[k])
        trace_boundingBox(image, mask, color=colors[k])

ax.imshow(image) 
plt.show()


#### 3.3. Mask inlay

This third and last example will consist in coloring the part of the image corresponding to the mask. This action is based on a threshold pixel value.

In our example we want to color the clouds (which are white) localized under the mask. The idea is to create a mask by isolating the area of the image corresponding to the original mask, convert it to black and white and then apply a filter on the value of the pixels. Then all we have to do is to color our new mask as we did in the first example.

In [ ]:
def cloudInColor(image : np.ndarray, 
                 mask : np.ndarray, 
                 color : tuple = (0,0,255),
                 alpha : float = 0.7, 
                 threshold : int = 90) -> np.ndarray:
    """
    Draw a bounding box on image

     Parameter
     ----------
     image : image on which we want to colorize parts 
     mask  : mask to process
     color : color we want to use to colorize image
     alpha : opacity coefficient
     threshold : pixel value threshold to apply
     
     Return
     ----------
     np.array : result of layering
    """    
    imZone = cv2.bitwise_and(image, image, mask=mask)
    image_gray = cv2.cvtColor(imZone, cv2.COLOR_RGB2GRAY)
    (thresh, blackAndWhiteImage) = cv2.threshold(image_gray, 
                                                 threshold, 
                                                 255, 
                                                 cv2.THRESH_BINARY)
    inlay = maskInColor(image, blackAndWhiteImage, color=color, alpha=alpha)
    return inlay

The result is rather aesthetic, I'll let you judge by yourself:

In [ ]:

fig, ax = plt.subplots()

image = np.copy(image_start)         #reset the working image
for k in range(4):                   #We have 4 classes in this dataset
    rle = rles[k]                    #initialize the current RLE code
    if not isinstance(rle, float):   #it's not a 'NaN' RLE
        mask = rleToMask(rles[k])
        image = cloudInColor(image, mask, color=colors[k], alpha=0.7, threshold=90)

ax.imshow(image) 
plt.show()


Hope these few functions will be helpful for someone ...